
# Fase 4 - Capítulo 1: Automação e Inteligência na Farmtech Solution 

**Aluno:** Vinicius Burchert Vilas Boas  
**RM:** 565395

**Aluno:** Felipe Balthazar de Almeida   
**RM:** 562434

**Aluno:** Fernando Gomes da Silva  
**RM:** 561534

**Aluno:** Guilherme Urbinatti  
**RM:** 565203

---


# SISTEMA DE IRRIGAÇÃO AUTOMATIZADO COM MACHINE LEARNING
Neste projeto, usamos a biblioteca Scikit-learn para treinar um modelo que prevê a necessidade de irrigação com base em dados reais de umidade, chuva, temperatura e nutrientes.

---


In [21]:

# Importação das bibliotecas

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import joblib

# 2. Carregamento dos dados

df = pd.read_csv("Atividade_Cap_14_produtos_agricolas.csv")

# Exibe as 5 primeiras linhas
print(df.head())


    N   P   K  temperature   humidity        ph    rainfall label
0  90  42  43    20.879744  82.002744  6.502985  202.935536  rice
1  85  58  41    21.770462  80.319644  7.038096  226.655537  rice
2  60  55  44    23.004459  82.320763  7.840207  263.964248  rice
3  74  35  40    26.491096  80.158363  6.980401  242.864034  rice
4  78  42  42    20.130175  81.604873  7.628473  262.717340  rice



## Sistema de Iriigação

Criação da variável alvo


In [22]:

# A coluna 'precisa_irrigar' é baseada na seguinte regra:
# - Umidade < 40
# - Chuva < 50
# - Temperatura > 28
# Se todas essas condições forem verdadeiras, então precisa irrigar (1), senão (0)

df['precisa_irrigar'] = ((df['humidity'] < 40) & 
                         (df['rainfall'] < 50) & 
                         (df['temperature'] > 28)).astype(int)

# Visualiza parte do DataFrame com a nova coluna
print(df[['temperature', 'humidity', 'rainfall', 'precisa_irrigar']].head())

   temperature   humidity    rainfall  precisa_irrigar
0    20.879744  82.002744  202.935536                0
1    21.770462  80.319644  226.655537                0
2    23.004459  82.320763  263.964248                0
3    26.491096  80.158363  242.864034                0
4    20.130175  81.604873  262.717340                0



##  Separação das variáveis


In [23]:
# X (features) são as entradas do modelo
# y (target) é o valor que queremos prever
X = df[['temperature', 'humidity', 'rainfall', 'N', 'P', 'K']]
y = df['precisa_irrigar']

## Escalonamento dos dados



In [24]:
# O StandardScaler normaliza os dados para média 0 e desvio padrão 1
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


## Divisão entre treino e teste



In [25]:
# 80% dos dados são usados para treino e 20% para teste
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Treinamento do modelo

In [26]:

# Usamos uma floresta aleatória (RandomForest), que é robusta e eficaz
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

RandomForestClassifier(random_state=42)


##  Avaliação do modelo





In [27]:
# Previsão nos dados de teste e relatório de desempenho
y_pred = modelo.predict(X_test)
print("Relatório de Classificação:\n")
print(classification_report(y_test, y_pred))

Relatório de Classificação:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       440

    accuracy                           1.00       440
   macro avg       1.00      1.00      1.00       440
weighted avg       1.00      1.00      1.00       440



# Salvamento do modelo e scaler

In [28]:
# Salvamos os objetos para uso futuro (ex: com Streamlit ou ESP32)
joblib.dump(modelo, 'modelo_irrigacao.pkl')
joblib.dump(scaler, 'scaler_irrigacao.pkl')

['scaler_irrigacao.pkl']

# Simulação com novo dado

In [30]:
# Simulação com novo dado do sensor
# Exemplo de entrada vinda do sensor:
# Temperatura (°C), Umidade (%), Potássio (0 ou 1), Fósforo (0 ou 1), Bomba (0 ou 1), pH

novo_dado_sensor = pd.DataFrame([{
    'temperature': 30.5,     # Temperatura em graus Celsius
    'humidity': 37.2,        # Umidade do solo
    'rainfall': 10.0,        # Chuva simulada (valor fixo, caso sensor não forneça)
    'N': 70,                 # Nitrogênio (mantido como estava)
    'P': 1,                  # Fósforo: 1 = presente
    'K': 1,                  # Potássio: 1 = presente
    # Os campos 'PH' e 'bomba' podem ser usados para exibição no frontend
    'PH': 6.2,
    'bomba': 1
}])

# Carrega o modelo e o scaler
modelo = joblib.load('modelo_irrigacao.pkl')
scaler = joblib.load('scaler_irrigacao.pkl')

# Apenas as colunas usadas no treinamento
colunas_utilizadas = ['temperature', 'humidity', 'rainfall', 'N', 'P', 'K']
novo_dado_scaled = scaler.transform(novo_dado_sensor[colunas_utilizadas])

# Faz a previsão
previsao = modelo.predict(novo_dado_scaled)

# Mostra o resultado
if previsao[0] == 1:
    print("⚠️ Necessário irrigar agora.")
else:
    print("✅ Irrigação não necessária no momento.")


✅ Irrigação não necessária no momento.
